Variáveis para os cálclulos:

x = dias de uso – considerando apontamentos/inventário 

w = variação número de apontamentos comparado ao mês anterior  

k = programações 

q = recursos 

j =  RDOs 

In [1]:
from tqdm.notebook import tqdm
from django.db.models import Count
from datetime import datetime, timedelta
import calendar

In [2]:
road_companies = ['Neovia Planalto Sul',
                 'Sinalizavia (Municípios)',
                 'Pavicolet (Amapá)',
                 'Pavicolet (Amazonas)',
                 'Arteris Planalto Sul - Conserva',
                 'Entrevias',
                 'Arteris Litoral Sul - Conserva',
                 'Arteris Régis Bittencourt - Conserva',
                 'Arteris Fluminense - Conserva',
                 'Arteris Fernão Dias - Conserva', 
                 'ELO Ecovias dos Imigrantes',
                 'ELO Rodovias dos Tamoios',
                 'Neovia Diamantino']

In [57]:
# road_companies = [x for x in Company.objects.filter(active=True).values_list('name',flat=True).order_by('name')]

In [3]:
today = datetime.now()-timedelta(days=10)
last_month = today - timedelta(days=30)
months = [last_month.month,today.month]
today,last_month,months

(datetime.datetime(2024, 3, 22, 10, 32, 1, 785015),
 datetime.datetime(2024, 2, 21, 10, 32, 1, 785015),
 [2, 3])

In [4]:
last_days = [calendar.monthrange(today.year, mes)[1] for mes in months]
last_days

[29, 31]

# x = dias de uso

In [5]:
def calculo_dias_de_uso(porcentagem):
    if porcentagem < 25:
        return 0
    elif 25 <= porcentagem < 65:
        return 0.3
    elif 65 <= porcentagem < 80:
        return 0.65
    else:
        return 1

In [6]:
def x(months, company):
    porcen_dias = []

    for x, month in enumerate(months):
        total_days_in_month = last_days[x]
        qtd_dias = []

        for day in range(1, total_days_in_month + 1):
            qtd_dias.append(
                Reporting.objects.filter(
                    company__name=company, 
                    updated_at__year=today.year, 
                    updated_at__month=month,
                    updated_at__day=day
                ).exclude(
                    created_by__username='rlcs'
                ).exclude(
                    created_by__username='neoviacs'
                ).exclude(
                    created_by__username='arteriscs'
                ).only('uuid').count()
            )

        # Calcula a quantidade de dias com apontamento
        quantity = sum(1 for qtd in qtd_dias if qtd != 0)
        # calcula a porcentagem de dias de uso, em relação ao total de dias no mês
        porcentagem = quantity / total_days_in_month * 100
        # o resultado é definido de acordo com as 'regras' do calculo_dias_de_uso
        porcentagem_dias_de_uso = calculo_dias_de_uso(porcentagem) # comentar pra ver a porcentagem sem o cálculo
        porcen_dias.append(porcentagem_dias_de_uso)

    return porcen_dias 

In [7]:
resultados_por_empresa = {}

for company in tqdm(road_companies, desc='Empresas'):
    resultados_por_empresa[company] = x(months, company)

for company, resultados in resultados_por_empresa.items():
    for index, porcentagem_dias_de_uso in enumerate(resultados):
        print(f"Nota da unidade {company}, mês {months[index]}: {porcentagem_dias_de_uso}")

Empresas:   0%|          | 0/13 [00:00<?, ?it/s]

Nota da unidade Neovia Planalto Sul, mês 2: 0.3
Nota da unidade Neovia Planalto Sul, mês 3: 0.3
Nota da unidade Sinalizavia (Municípios), mês 2: 0
Nota da unidade Sinalizavia (Municípios), mês 3: 0.3
Nota da unidade Pavicolet (Amapá), mês 2: 0.3
Nota da unidade Pavicolet (Amapá), mês 3: 1
Nota da unidade Pavicolet (Amazonas), mês 2: 1
Nota da unidade Pavicolet (Amazonas), mês 3: 1
Nota da unidade Arteris Planalto Sul - Conserva, mês 2: 0.65
Nota da unidade Arteris Planalto Sul - Conserva, mês 3: 0.3
Nota da unidade Entrevias, mês 2: 1
Nota da unidade Entrevias, mês 3: 1
Nota da unidade Arteris Litoral Sul - Conserva, mês 2: 1
Nota da unidade Arteris Litoral Sul - Conserva, mês 3: 1
Nota da unidade Arteris Régis Bittencourt - Conserva, mês 2: 0.65
Nota da unidade Arteris Régis Bittencourt - Conserva, mês 3: 0.65
Nota da unidade Arteris Fluminense - Conserva, mês 2: 0
Nota da unidade Arteris Fluminense - Conserva, mês 3: 0
Nota da unidade Arteris Fernão Dias - Conserva, mês 2: 0.65
Nota 

# w = variação número de apontamentos comparado ao mês anterior  

In [8]:
def calcular_variacao(atual, anterior):
    if anterior == 0:
        return 1, "Aumento de 100%"
    # Quando não há apontamentos no mês anterior, é  considerado como aumento de 100%

    variacao = (atual - anterior) / anterior * 100

    if variacao >= 0:
        return 1, "Positivo"  # Aumento ou igual é considerado Postivo
    elif -15 <= variacao < 0:
        return 0.7, "Regular"
    elif -25 <= variacao < -15:
        return 0.35, "Mediano"
    else:
        return 0, "Atenção"

In [10]:
# Cálculo de w
w = []

for company in tqdm(road_companies):
    atual_count = Reporting.objects.filter(
        company__name=company,
        updated_at__year=today.year,
        updated_at__month=today.month
    ).exclude(
        created_by__username='rlcs'
    ).exclude(
        created_by__username='neoviacs'
    ).exclude(
        created_by__username='arteriscs'
    ).only('uuid').count()

    anterior_count = Reporting.objects.filter(
        company__name=company,
        updated_at__year=last_month.year,
        updated_at__month=last_month.month
    ).exclude(
        created_by__username='rlcs'
    ).exclude(
        created_by__username='neoviacs'
    ).exclude(
        created_by__username='arteriscs'
    ).only('uuid').count()

    atual = int(atual_count)
    anterior = int(anterior_count)
    print(company,atual,anterior)
    # Calcular a variação
    variacao, categoria = calcular_variacao(atual, anterior)
    w.append((company, variacao))

    print(company, variacao)
    print()

  0%|          | 0/13 [00:00<?, ?it/s]

Neovia Planalto Sul 120 76
Neovia Planalto Sul 1

Sinalizavia (Municípios) 15 11
Sinalizavia (Municípios) 1

Pavicolet (Amapá) 492 102
Pavicolet (Amapá) 1

Pavicolet (Amazonas) 612 695
Pavicolet (Amazonas) 0.7

Arteris Planalto Sul - Conserva 656 381
Arteris Planalto Sul - Conserva 1

Entrevias 9422 8270
Entrevias 1

Arteris Litoral Sul - Conserva 32979 3705
Arteris Litoral Sul - Conserva 1

Arteris Régis Bittencourt - Conserva 3560 4770
Arteris Régis Bittencourt - Conserva 0

Arteris Fluminense - Conserva 7 18
Arteris Fluminense - Conserva 0

Arteris Fernão Dias - Conserva 1232 1114
Arteris Fernão Dias - Conserva 1

ELO Ecovias dos Imigrantes 309 472
ELO Ecovias dos Imigrantes 0

ELO Rodovias dos Tamoios 375 605
ELO Rodovias dos Tamoios 0

Neovia Diamantino 108 92
Neovia Diamantino 1



# k = programações

In [27]:
def calcular_valor_programacoes(programacoes):
    # Obtém o número de programações
    num_programacoes = programacoes.count()
    
    if num_programacoes >= 4:
        return 1
    elif 2 <= num_programacoes <= 3:
        return 0.5
    else:
        return 0

In [28]:
def k(road_companies, today, last_month,):
    result = {}

    for company in tqdm(road_companies):
        last = Job.history.filter(
            history_date__month=last_month.month,
            history_date__year=last_month.year,
            company__name=company,
            history_type='+'
        )

        valor_mes_anterior = calcular_valor_programacoes(last)

        print(f'{company}: Nota programações mês anterior = {valor_mes_anterior}')

        atual = Job.history.filter(
            history_date__month=today.month,
            history_date__year=today.year,
            company__name=company,
            history_type='+'
        )

        valor_atual = calcular_valor_programacoes(atual)

        print(f'{company}: Nota programações mês atual = {valor_atual}')

        result[company] = (valor_atual, valor_mes_anterior)

    return result

# Chamando a função k e armazenando os resultados
k_result = k(road_companies, today, last_month)

  0%|          | 0/13 [00:00<?, ?it/s]

Neovia Planalto Sul: Nota programações mês anterior = 0
Neovia Planalto Sul: Nota programações mês atual = 0
Sinalizavia (Municípios): Nota programações mês anterior = 1
Sinalizavia (Municípios): Nota programações mês atual = 0.5
Pavicolet (Amapá): Nota programações mês anterior = 0.5
Pavicolet (Amapá): Nota programações mês atual = 1
Pavicolet (Amazonas): Nota programações mês anterior = 1
Pavicolet (Amazonas): Nota programações mês atual = 1
Arteris Planalto Sul - Conserva: Nota programações mês anterior = 1
Arteris Planalto Sul - Conserva: Nota programações mês atual = 1
Entrevias: Nota programações mês anterior = 1
Entrevias: Nota programações mês atual = 1
Arteris Litoral Sul - Conserva: Nota programações mês anterior = 1
Arteris Litoral Sul - Conserva: Nota programações mês atual = 1
Arteris Régis Bittencourt - Conserva: Nota programações mês anterior = 1
Arteris Régis Bittencourt - Conserva: Nota programações mês atual = 1
Arteris Fluminense - Conserva: Nota programações mês ant

# q = recursos

In [29]:
from tqdm import tqdm

def q(road_companies, today, last_month):
    result = []

    print("Nota de recursos no mês atual:")
    for company in tqdm(road_companies):
        contador = 0
        total_equipes = ProcedureResource.objects.filter(
            firm__company__name=company,
            creation_date__month=today.month,
            creation_date__year=today.year
        ).count()

        # Verificando se foram criados recursos no mês atual
        porcentagem_atual = 1 if total_equipes > 0 else 0
        result.append((company, porcentagem_atual))
        print(f'{company}: Nota recursos = {porcentagem_atual}')

    print()

    # Calculando a porcentagem para o mês anterior
    print("Nota de recursos no mês anterior:")
    for company in tqdm(road_companies):
        contador = 0
        total_equipes = ProcedureResource.objects.filter(
            firm__company__name=company,
            creation_date__month=last_month.month,
            creation_date__year=last_month.year
        ).count()

        # Verificando se foram criados recursos no mês anterior
        porcentagem_anterior = 1 if total_equipes > 0 else 0
        result.append((company, porcentagem_anterior))
        print(f'{company}: Nota recursos = {porcentagem_anterior}')

    return result

# Chamando a função q e armazenando os resultados
q_result = q(road_companies, last_month, today)

Nota de recursos no mês atual:


  0%|                                                    | 0/13 [00:00<?, ?it/s]

Neovia Planalto Sul: Nota recursos = 0
Sinalizavia (Municípios): Nota recursos = 0
Pavicolet (Amapá): Nota recursos = 1
Pavicolet (Amazonas): Nota recursos = 1
Arteris Planalto Sul - Conserva: Nota recursos = 0
Entrevias: Nota recursos = 0
Arteris Litoral Sul - Conserva: Nota recursos = 1
Arteris Régis Bittencourt - Conserva: Nota recursos = 0
Arteris Fluminense - Conserva: Nota recursos = 0


100%|██████████████████████████████████████████| 13/13 [00:00<00:00, 324.67it/s]


Arteris Fernão Dias - Conserva: Nota recursos = 0
ELO Ecovias dos Imigrantes: Nota recursos = 0
ELO Rodovias dos Tamoios: Nota recursos = 0
Neovia Diamantino: Nota recursos = 0

Nota de recursos no mês anterior:


100%|██████████████████████████████████████████| 13/13 [00:00<00:00, 324.34it/s]

Neovia Planalto Sul: Nota recursos = 0
Sinalizavia (Municípios): Nota recursos = 0
Pavicolet (Amapá): Nota recursos = 1
Pavicolet (Amazonas): Nota recursos = 0
Arteris Planalto Sul - Conserva: Nota recursos = 0
Entrevias: Nota recursos = 0
Arteris Litoral Sul - Conserva: Nota recursos = 1
Arteris Régis Bittencourt - Conserva: Nota recursos = 0
Arteris Fluminense - Conserva: Nota recursos = 0
Arteris Fernão Dias - Conserva: Nota recursos = 0
ELO Ecovias dos Imigrantes: Nota recursos = 0
ELO Rodovias dos Tamoios: Nota recursos = 0
Neovia Diamantino: Nota recursos = 0


# j = RDOs

In [30]:
def calcular_valor_RDO(porcentagem):
    if porcentagem >= 50:
        return 1
    elif 15 <= porcentagem < 50:
        return 0.5
    else:
        return 0

In [31]:
def j(road_companies, today, last_month):
    valor_RDO_atual_por_empresa = {}
    valor_RDO_anterior_por_empresa = {}

    print("Porcentagem de equipes com RDOs criados no mês atual:")
    for company in tqdm(road_companies):
        contador = 0
        firms = Firm.objects.filter(company__name=company)
        total_equipes = firms.count()

        for firm in firms:
            if firm.firm_multiple_daily_report.filter(
                Q(date__month=today.month),
                Q(date__year=today.year)
            ).exists():
                contador += 1

        porcentagem_atual = (contador / total_equipes) * 100 if total_equipes != 0 else 0
        valor_RDO_atual = calcular_valor_RDO(porcentagem_atual)
        print(f'Empresa: {company}, Nota RDO: {valor_RDO_atual}')
        valor_RDO_atual_por_empresa[company] = valor_RDO_atual

    print()

    # Calculando a porcentagem para o mês anterior
    print("Porcentagem de equipes com RDOs criados no mês anterior:")
    for company in tqdm(road_companies):
        contador = 0
        firms = Firm.objects.filter(company__name=company)
        total_equipes = firms.count()

        for firm in firms:
            if firm.firm_multiple_daily_report.filter(
                Q(date__month=last_month.month),
                Q(date__year=last_month.year)
            ).exists():
                contador += 1

        porcentagem_anterior = (contador / total_equipes) * 100 if total_equipes != 0 else 0
        valor_RDO_anterior = calcular_valor_RDO(porcentagem_anterior)
        print(f'Empresa: {company}, Nota RDO: {valor_RDO_anterior}')
        valor_RDO_anterior_por_empresa[company] = valor_RDO_anterior

    return valor_RDO_atual_por_empresa, valor_RDO_anterior_por_empresa

valor_RDO_atual_por_empresa, valor_RDO_anterior_por_empresa = j(road_companies, today, last_month)

Porcentagem de equipes com RDOs criados no mês atual:


  0%|                                                    | 0/13 [00:00<?, ?it/s]

Empresa: Neovia Planalto Sul, Nota RDO: 0
Empresa: Sinalizavia (Municípios), Nota RDO: 0
Empresa: Pavicolet (Amapá), Nota RDO: 0.5


 31%|█████████████▌                              | 4/13 [00:00<00:00, 35.64it/s]

Empresa: Pavicolet (Amazonas), Nota RDO: 0
Empresa: Arteris Planalto Sul - Conserva, Nota RDO: 0.5
Empresa: Entrevias, Nota RDO: 0.5
Empresa: Arteris Litoral Sul - Conserva, Nota RDO: 0.5


...

Empresa: Neovia Planalto Sul, Nota RDO: 0
Empresa: Sinalizavia (Municípios), Nota RDO: 0
Empresa: Pavicolet (Amapá), Nota RDO: 0.5
Empresa: Pavicolet (Amazonas), Nota RDO: 0
Empresa: Arteris Planalto Sul - Conserva, Nota RDO: 0.5
Empresa: Entrevias, Nota RDO: 0.5
Empresa: Arteris Litoral Sul - Conserva, Nota RDO: 1


 62%|███████████████████████████                 | 8/13 [00:00<00:00,  8.39it/s]

Empresa: Arteris Régis Bittencourt - Conserva, Nota RDO: 0.5
Empresa: Arteris Fluminense - Conserva, Nota RDO: 0


100%|███████████████████████████████████████████| 13/13 [00:01<00:00, 10.53it/s]

Empresa: Arteris Fernão Dias - Conserva, Nota RDO: 0.5
Empresa: ELO Ecovias dos Imigrantes, Nota RDO: 0
Empresa: ELO Rodovias dos Tamoios, Nota RDO: 0
Empresa: Neovia Diamantino, Nota RDO: 0


# HS (básico) 

Exemplo:
    Porcentagem de uso 0.3
    Variação de apontamentos: 1
Exemplo cálculo:
    hs_basico = 50 * 0.3 + 50 * 1
          = 15 + 50
          = 65

In [32]:
def calcular_hs_basico(resultados_por_empresa, w):
    hs_basico = {}
    hs_export_basico = {}

    for company, variacao in w:
        hs_basico[company] = []
        hs_export_basico[company] = []

        for i, month in enumerate(months):
            porcentagem_dias_de_uso = resultados_por_empresa[company][i]
            variacao_apontamentos = variacao
            
            hs_basico_value = 50 * porcentagem_dias_de_uso + 50 * variacao_apontamentos
            
            company_str = str(company).ljust(30)
            month_str = str(month).ljust(10)
            print(f"Empresa: {company_str} | Mês: {month_str} | Porcentagem de dias de uso: {porcentagem_dias_de_uso} | Variação de apontamentos: {variacao_apontamentos}")
            print(f"HS Básico para a empresa {company} no mês {month}: {hs_basico_value}\n")

            hs_basico[company].append(hs_basico_value)
            hs_export_basico[company].append(hs_basico_value)

    return hs_basico, hs_export_basico

hs_basico, hs_export_basico = calcular_hs_basico(resultados_por_empresa, w)

Empresa: Neovia Planalto Sul            | Mês: 2          | Porcentagem de dias de uso: 0.3 | Variação de apontamentos: 1
HS Básico para a empresa Neovia Planalto Sul no mês 2: 65.0

Empresa: Neovia Planalto Sul            | Mês: 3          | Porcentagem de dias de uso: 0.3 | Variação de apontamentos: 1
HS Básico para a empresa Neovia Planalto Sul no mês 3: 65.0

Empresa: Sinalizavia (Municípios)       | Mês: 2          | Porcentagem de dias de uso: 0 | Variação de apontamentos: 1
HS Básico para a empresa Sinalizavia (Municípios) no mês 2: 50

Empresa: Sinalizavia (Municípios)       | Mês: 3          | Porcentagem de dias de uso: 0 | Variação de apontamentos: 1
HS Básico para a empresa Sinalizavia (Municípios) no mês 3: 50

Empresa: Pavicolet (Amapá)              | Mês: 2          | Porcentagem de dias de uso: 0.3 | Variação de apontamentos: 1
HS Básico para a empresa Pavicolet (Amapá) no mês 2: 65.0

Empresa: Pavicolet (Amapá)              | Mês: 3          | Porcentagem de dias de us

# HS (intermediário) 

Exemplo:
    Porcentagem de uso 0.3
    Variação de apontamentos: 1
    Nota programações: 0
Exemplo de cálculo:
    hs_intermediario = 34 * 0.3 + 34 * 1 + 32 * 0
                     = 10.2 + 34 + 0
                     = 44.2

In [33]:
def calcular_hs_intermediario(resultados_por_empresa, w, k_result):
    hs_intermediario = {}
    hs_export_intermediario = {}

    for company, variacao in w:
        hs_intermediario[company] = []
        hs_export_intermediario[company] = []

        for i, month in enumerate(months):
            porcentagem_dias_de_uso = resultados_por_empresa[company][i]
            variacao_apontamentos = variacao
            valor_programacoes = k_result[company][i]
            
            hs_intermediario_value = 34 * porcentagem_dias_de_uso + 34 * variacao_apontamentos + 32 * valor_programacoes
            
            company_str = str(company).ljust(30)
            month_str = str(month).ljust(10)
            print(f"Empresa: {company_str} | Mês: {month_str} | Porcentagem de dias de uso: {porcentagem_dias_de_uso} | Variação de apontamentos: {variacao_apontamentos} | Valor de programações: {valor_programacoes}")
            print(f"HS Intermediário para a empresa {company} no mês {month}: {hs_intermediario_value}\n")

            hs_intermediario[company].append(hs_intermediario_value)
            hs_export_intermediario[company].append(hs_intermediario_value)

    return hs_intermediario, hs_export_intermediario

hs_intermediario, hs_export_intermediario = calcular_hs_intermediario(resultados_por_empresa, w, k_result)

Empresa: Neovia Planalto Sul            | Mês: 2          | Porcentagem de dias de uso: 0.3 | Variação de apontamentos: 1 | Valor de programações: 0
HS Intermediário para a empresa Neovia Planalto Sul no mês 2: 44.2

Empresa: Neovia Planalto Sul            | Mês: 3          | Porcentagem de dias de uso: 0.3 | Variação de apontamentos: 1 | Valor de programações: 0
HS Intermediário para a empresa Neovia Planalto Sul no mês 3: 44.2

Empresa: Sinalizavia (Municípios)       | Mês: 2          | Porcentagem de dias de uso: 0 | Variação de apontamentos: 1 | Valor de programações: 0.5
HS Intermediário para a empresa Sinalizavia (Municípios) no mês 2: 50.0

Empresa: Sinalizavia (Municípios)       | Mês: 3          | Porcentagem de dias de uso: 0 | Variação de apontamentos: 1 | Valor de programações: 1
HS Intermediário para a empresa Sinalizavia (Municípios) no mês 3: 66

Empresa: Pavicolet (Amapá)              | Mês: 2          | Porcentagem de dias de uso: 0.3 | Variação de apontamentos: 1 | Va

# HS (avançado) 

Exemplo:
    Porcentagem de uso: 0.3
    Variação de apontamentos: 1 (Positivo)
    Nota programações: 0
    Nota Recursos = 0
    Nota RDO: 0
Exemplo de cálculo:
    hs_avancado_value = 20 * (0.3 + 1 + 0 + 0 + 0)
                      = 20 * 1.3
                      = 26

In [34]:
def calcular_hs_avancado(resultados_por_empresa, w, k_result, q, valor_RDO_atual_por_empresa):
    hs_avancado = {}
    hs_export_avancado = {}

    for company, variacao in w:
        hs_avancado[company] = []
        hs_export_avancado[company] = []

        for i, month in enumerate(months):
            porcentagem_dias_de_uso = resultados_por_empresa[company][i]
            variacao_apontamentos = variacao
            valor_programacoes = k_result[company][i] 
            nota_recursos = q[i][1] 
            nota_rdo = valor_RDO_atual_por_empresa.get(company, 0)  
            
            company_str = str(company).ljust(30)
            month_str = str(month).ljust(10)
            
            print(f"Empresa: {company_str} | Mês: {month_str} | Porcentagem de dias de uso: {porcentagem_dias_de_uso} | Variação de apontamentos: {variacao_apontamentos} | Valor de programações: {valor_programacoes} | Nota de recursos: {nota_recursos} | Nota RDO: {nota_rdo}")

            hs_avancado_value = 20 * porcentagem_dias_de_uso + 20 * variacao_apontamentos + 20 * valor_programacoes + 20 * nota_recursos + 20 * nota_rdo
            print(f"HS Avançado para a empresa {company} no mês {month}: {hs_avancado_value}\n")

            hs_avancado[company].append(hs_avancado_value)
            hs_export_avancado[company].append(hs_avancado_value)


    return hs_avancado, hs_export_avancado

hs_avancado, hs_export_avancado = calcular_hs_avancado(resultados_por_empresa, w, k_result, q_result, valor_RDO_atual_por_empresa)

Empresa: Neovia Planalto Sul            | Mês: 2          | Porcentagem de dias de uso: 0.3 | Variação de apontamentos: 1 | Valor de programações: 0 | Nota de recursos: 0 | Nota RDO: 0
HS Avançado para a empresa Neovia Planalto Sul no mês 2: 26.0

Empresa: Neovia Planalto Sul            | Mês: 3          | Porcentagem de dias de uso: 0.3 | Variação de apontamentos: 1 | Valor de programações: 0 | Nota de recursos: 0 | Nota RDO: 0
HS Avançado para a empresa Neovia Planalto Sul no mês 3: 26.0

Empresa: Sinalizavia (Municípios)       | Mês: 2          | Porcentagem de dias de uso: 0 | Variação de apontamentos: 1 | Valor de programações: 0.5 | Nota de recursos: 0 | Nota RDO: 0
HS Avançado para a empresa Sinalizavia (Municípios) no mês 2: 30.0

Empresa: Sinalizavia (Municípios)       | Mês: 3          | Porcentagem de dias de uso: 0 | Variação de apontamentos: 1 | Valor de programações: 1 | Nota de recursos: 0 | Nota RDO: 0
HS Avançado para a empresa Sinalizavia (Municípios) no mês 3: 40

Em